# Data Science Workbook

## Glue Calalog Context

In [3]:
from config.spark import SparkIcebergConfig
spark = SparkIcebergConfig.create_session(config_type="glue")

25/09/23 06:12:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|         glue|
|spark_catalog|
+-------------+



In [9]:
spark.sql("SHOW NAMESPACES").show()

+---------+
|namespace|
+---------+
|   bronze|
|     demo|
|     gold|
|   silver|
+---------+



In [9]:
%%sql
SELECT *from glue.silver.weatherhistory limit 1;

formatted_date,summary,precip_type,temperature_c,apparent_temperature_c,humidity,wind_speed_kmh,wind_bearing_degrees,visibility_km,loud_cover,pressure_mb,daily_summary
2008-05-31 22:00:00,Mostly Cloudy,rain,19.777777777777775000,19.777777777777775000,0.690000000000000000,6.005300000000000000,78.000000000000000000,15.826300000000002000,0.0,None,Mostly cloudy until night.


In [1]:
%%sql
SELECT *from glue.gold.weatherhistory limit 1;

25/09/18 19:58:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
                                                                                

formatted_date,summary,precip_type,temperature_c,apparent_temperature_c,humidity,wind_speed_kmh,wind_bearing_degrees,visibility_km,loud_cover,pressure_mb,daily_summary
2008-05-31 22:00:00,Mostly Cloudy,rain,19.777777777777775000,19.777777777777775000,0.690000000000000000,6.005300000000000000,78.000000000000000000,15.826300000000002000,0.0,None,Mostly cloudy until night.


In [10]:
data = [("Alice", 1), ("Bob", 2), ("Charlie", 3)]
df = spark.createDataFrame(data, ["Name", "ID"])
df.show()

+-------+---+
|   Name| ID|
+-------+---+
|  Alice|  1|
|    Bob|  2|
|Charlie|  3|
+-------+---+



In [12]:
df.writeTo("glue.bronze.test_new").createOrReplace()

In [13]:
spark.sql("select * from glue.bronze.test_new").show()

[Stage 13:>                                                         (0 + 1) / 1]

+-------+---+
|   Name| ID|
+-------+---+
|  Alice|  1|
|    Bob|  2|
|Charlie|  3|
+-------+---+



## Query Analysis

In [9]:
%%sql
select * from glue.silver.weatherhistory limit 1;

formatted_date,summary,precip_type,temperature_c,apparent_temperature_c,humidity,wind_speed_kmh,wind_bearing_degrees,visibility_km,loud_cover,pressure_mb,daily_summary
2008-05-31 22:00:00,Mostly Cloudy,rain,19.777777777777775000,19.777777777777775000,0.690000000000000000,6.005300000000000000,78.000000000000000000,15.826300000000002000,0.0,None,Mostly cloudy until night.


In [8]:
spark.sql("""
SELECT `zip_code`,
       COUNT(*)               AS record_count,
       MAX(temp)              AS max_temp,
       COUNT(DISTINCT dt_iso) AS distinct_timestamps
FROM glue.silver.weatherhistory
GROUP BY `zip_code`
ORDER BY `zip_code`
LIMIT 200""");

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `zip_code` cannot be resolved. Did you mean one of the following? [`loud_cover`, `precip_type`, `summary`, `humidity`, `pressure_mb`].; line 2 pos 7;
'GlobalLimit 200
+- 'LocalLimit 200
   +- 'Sort ['zip_code ASC NULLS FIRST], true
      +- 'Aggregate ['zip_code], ['zip_code, count(1) AS record_count#87L, 'MAX('temp) AS max_temp#88, 'COUNT(distinct 'dt_iso) AS distinct_timestamps#89]
         +- SubqueryAlias glue.silver.weatherhistory
            +- RelationV2[formatted_date#90, summary#91, precip_type#92, temperature_c#93, apparent_temperature_c#94, humidity#95, wind_speed_kmh#96, wind_bearing_degrees#97, visibility_km#98, loud_cover#99, pressure_mb#100, daily_summary#101] glue.silver.weatherhistory glue.silver.weatherhistory
